In [1]:
from find_source import full_html_and_txt, make_catalog
import glob
import json

KeyboardInterrupt: 

In [ ]:
folders = []
for folder in glob.glob('/reduction/karto/SMA/polaris_test/*'):
    print(folder)
    folders.append(folder)

In [ ]:
full_html_and_txt(folders[0])
folders = folders[1:]